# 要約 
このJupyter Notebookは、Kaggleコンペティション「LLM 20 Questions」に向けたスターターコードです。特に、`phi3`言語モデルを使用して、ゲームの基礎的な機能を実装することを目的としています。ノートブックは、Pythonパッケージ「rigging」を使用してエージェントを構築し、`vLLM`を用いてモデルをローカルでホスティングします。この取り組みは、20の質問ゲームのための「質問者」と「回答者」モデルを作成するためのものです。

### アプローチと手法
- **riggingライブラリ**: これは軽量のLLMインタラクションフレームワークで、異なるLLMモデルを簡単に切り替え、クエリパイプラインを設計するのを助けます。このNotebookでは、`rigging`を使用してエージェントの質問と応答の管理を行っています。
  
- **vLLMライブラリ**: このライブラリは、モデルをローカルでホスティングするために使われます。ノートブックで`vLLM`サーバーを立ち上げる構成が含まれており、APIインターフェイスを介してモデルを利用できます。

- **データの準備**: Hugging Faceからモデルの重みをダウンロードし、Kaggleの秘密管理機能を利用してAPIトークンを安全に扱います。また、キーワードデータをパラケットファイルとして作成・管理します。キーワードについての質問を生成し、その回答を管理するためのデータフレームが作成されます。

- **エージェントの設計**: 属性を定義した`Observation`クラスと、質問・回答・推測を管理するためのパイプラインが設計されています。また、カテゴリーに基づいてキーワードをフィルタリングするためのシンプルな質問者エージェントが実装されています。

###ライブラリ
- **Pydantic**: 型注釈とデータの検証を行うために使用されています。
- **Numpy/Pandas**: データフレームを操作するためのライブラリ。質問履歴やキーワードデータの管理に用いられます。

このノートブックは、基本的な構成を提供し、特に質問の生成や推測、回答を通じて、20の質問の戦略を展開するための強力な基盤を築いています。さらに、LLMの能力を活かして、質問の内容を改善するための戦略的なアプローチが盛り込まれています。

---


# 用語概説 
以下に、Jupyter Notebookに登場するが初心者がつまずきそうな専門用語の簡単な解説を列挙します。

1. **Rigging**:
   - 言語モデル（LLM）とのインタラクションを行うためのフレームワーク。これにより、異なるLLMモデルへの接続や出力のチェック、再試行ロジックを簡単に実装できる。

2. **vLLM**:
   - LLMをローカルで実行するためのサービスで、高速なモデルホスティングを提供する。特に、リソースを効率的に使用して、モデルのインスタンスを生成・管理する。

3. **Pydantic**:
   - Pythonのデータバリデーション及び設定管理を行うためのライブラリ。特に、データの整合性を保つために型ヒントを使用し、簡潔なクラス定義で複雑なバリデーションを行うことができる。

4. **Subprocess**:
   - Pythonで外部コマンドを実行するためのモジュール。このモジュールを使用すると、Pythonプログラムからシェルコマンドを実行し、入力・出力を管理することができる。

5. **ChatPipeline**:
   - LLMに対する対話セッションを構築するための一連の手続き。特定のプロンプトに基づいて質問を行い、その応答を処理するプロセスを指す。

6. **Observation**:
   - ゲームのステータスや履歴を保持するためのデータ構造。質問や回答の履歴を管理し、エージェントが次の質問を決定するのに役立つ。

7. **XML形式**:
   - データを構造化するためのマークアップ言語。特に、LLMに対する特定の形式での出力や入力を定義するのに用いられる。

8. **Numpy**:
   - 数値計算を効率的に行うためのPythonライブラリ。特に多次元配列（ndarray）を操作するのに最適化されている。

9. **Streamlit**:
   - データアプリケーションを簡単に作成するためのフレームワーク。特に機械学習のモデルを迅速にデモンストレーションするために使用される。

10. **Latency**:
    - システムが要求に応じて反応するまでの時間。特にリアルタイムシステムや対話エージェントで重要な指標となる。

11. **Metadata**:
    - データについての情報（データのデータ）。例えば、データ作成日時、データソースなどが含まれることがある。

12. **Docker**:
    - ソフトウェアをコンテナ化して、開発・デプロイを行うためのプラットフォーム。環境構築を容易にし、異なるシステム間での移植性を高める。

13. **Git Submodules**:
    - 他のGitリポジトリを自身のリポジトリ内に含めるための機能。他のプロジェクトのコードを参照するのに役立つ。

14. **parquet形式**:
    - 列指向のデータファイルフォーマットで、特にビッグデータ処理の中で効率的にデータを保存・読み込むために利用される。

これらの用語は、ノートブックの内容および機械学習・深層学習の実務において重要な概念やツール、技術に関連しています。初心者がこれらを理解することで、プロジェクトや競技に参加する際の助けとなるでしょう。

---


# LLM 20 Questions スターター（Rigging [Phi3]を使用）

## このノートブックは、以前のノートブックの改訂版です。@bhanupmの要求に応じて、`llama3`モデルを`phi3`モデルに置き換えました。元のノートブックは[こちら](https://www.kaggle.com/code/robikscube/phi3-intro-to-rigging-for-llm-20-questions/)で見つけることができます。

私の初期テストでは、llama3バージョンの方がパフォーマンスが良いですが、より良いプロンプトを用いることで改善される可能性があります。

このスターターノートブックは、Pythonパッケージ「rigging」を使用して、コンペティションの基準提出を作成する方法を示しています。このセットアップでは、vLLMを使用して`phi3`量子化モデルを使用します。

## 更新 **2024年6月10日**
- rigging 2.0に対応するようにコードを更新しました
- プライベートリーダーボードではうまく機能しないことに注意しながら、既知のキーワードを利用する非LLM質問エージェントを含めました。回答エージェントはrigging経由でLLMを使用します。

## Riggingとは？

Riggingは、Pydantic XMLに基づいた軽量のLLMインタラクションフレームワークです。目的は、LLMをプロダクションパイプラインで利用する際にできるだけ簡単かつ効果的にすることです。Riggingは20の質問タスクに完全に適しており、以下のことができます：
1. 異なるバックエンドLLMモデルを簡単に切り替えることができます。
2. 期待される出力をチェックし、成功するまで再試行するLLMクエリパイプラインを設計できます。
3. 型ヒント、非同期サポート、Pydanticバリデーション、シリアル化など、現代的なPythonを活用しています。

こちらからリポジトリにスターを付けてください：https://github.com/dreadnode/rigging  
ドキュメントはここで読むことができます：https://rigging.dreadnode.io/

Riggingは[dreadnode](https://www.dreadnode.io/)によって構築され、維持されています。私たちは日々の作業でこれを使用しています。

例として、riggingパイプラインは次のようになります：
```{python}
chat = rg.get_generator('gpt-4o') \
    .chat(f"南アメリカのすべての国の名前で、Aで始まるものを提供してください {Answer.xml_tags()} タグ。") \
    .until_parsed_as(Answer) \
    .run() 
```

大半の主要なLLM APIとシームレスに生成器を作成することができるため、APIキーが環境変数として保存されている限り可能です。
```
export OPENAI_API_KEY=...
export TOGETHER_API_KEY=...
export TOGETHERAI_API_KEY=...
export MISTRAL_API_KEY=...
export ANTHROPIC_API_KEY=...
```

このコンペティションでは、モデルをローカルで実行する必要があり、幸いなことに、riggingはバックエンドでtransformersを使用してモデルを実行するサポートがあります。

# セットアップ

以下は、このノートブックのいくつかの設定です。ここでは：
- Hugging FaceおよびKaggleのためのシークレットトークンを読み込みます（オプション）
- 必要なパッケージをインストールします
- vLLMサーバーをテストするためのヘルパユーティリティスクリプトを作成します

このノートブックでは、Kaggleのシークレットを使用していくつかの隠れたトークンを使用しています。これはオプションであり、コードを実行するために必須ではありません。

In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

# Hugging Faceのトークンを格納するための変数
HF_TOKEN: str | None  = None
# Kaggleのキーを格納するための変数
KAGGLE_KEY: str | None = None
# Kaggleのユーザー名を格納するための変数
KAGGLE_USERNAME: str | None = None
    
try:
    # Kaggleの秘密からHugging Faceトークンを取得
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    # Kaggleの秘密からKaggleキーを取得
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    # Kaggleの秘密からユーザー名を取得
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    # もし何かエラーが発生した場合は何もしない（エラーを無視）
    pass

# 上記のコードは、Kaggleの秘密管理機能を利用して、必要なAPIキーやトークンを安全に取得するために使用されます。
# もし秘密が正常に取得できなかった場合、エラー処理として何も行わずにスクリプトの実行を続行します。

## パッケージのインストール
私たちは以下のパッケージをインストールします：
- [rigging](https://github.com/dreadnode/rigging)：このコンペティションのために私たちのLLMパイプラインを作成するために使用します。
- [vLLM](https://github.com/vllm-project/vllm)：モデルをローカルで独立したサービスとしてホスティングするために使用します。

また、[uv](https://github.com/astral-sh/uv)も使用します。これは、これらのパッケージをより迅速にインストールするのに役立ちます。

**注意:** これらのパッケージは`/kaggle/tmp/lib`ディレクトリにインストールします。これはコンペティションのセットアップの目的のためであり、後で提出用のzipファイルにこのパスのファイルを含める必要があります。また、vllmの依存関係を`/kaggle/tmp/srvlib`にもインストールします。

In [ ]:
# 依存関係（速度向上のためのuv）
# uvライブラリを使用してインストールの速度を向上させるために、指定のバージョンをインストールします。
!pip install uv==0.1.45

# riggingとkaggleライブラリを指定のターゲットにインストールします。
# --targetオプションを使用して特定のディレクトリにインストールします。
!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==2.0.0 \
    kaggle

# vllmとnumpyライブラリを別のターゲットにインストールします。
!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm==0.4.2 \
    numpy==1.26.4

# 上記のコードは、特定のターゲットディレクトリに依存関係をインストールするために使用されます。
# uvを使用することで、インストールプロセスが速くなり、効率的にライブラリを管理できます。

# LLMをローカルにダウンロード

このコンペティションでは、モデルの重みを含めたコードを提出する必要があるため、まず`snapshot_download`を使用してモデルの重みをダウンロードします。

私たちは`solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ`をダウンロードします。これは、コンペティションの要件を満たすのに十分小さな、アクティベーションアウェアの重み量子化バージョンのモデルです。

**注意**: 通常の状況でriggingを使用する場合、このステップは必要ありませんが、コンペティションのために重みを別にダウンロードして、提出用のzipファイルに含めることができるようにしています。

In [ ]:
# モデルをダウンロード

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

# モデルの格納先パスを設定
g_model_path = Path("/kaggle/tmp/model")
# もし指定したパスがすでに存在する場合は、その内容を削除
if g_model_path.exists():
    shutil.rmtree(g_model_path)
# 新しいモデル格納先のディレクトリを作成
g_model_path.mkdir(parents=True)

# Hugging Faceからモデルのスナップショットをダウンロード
snapshot_download(
    repo_id="rhysjones/Phi-3-mini-mango-1-llamafied",  # ダウンロードするモデルのリポジトリID
    ignore_patterns="original*",  # 無視するファイルパターン
    local_dir=g_model_path,  # モデル格納先のローカルディレクトリ
    local_dir_use_symlinks=False,  # シンボリックリンクを使用しない
    token=globals().get("HF_TOKEN", None)  # トークンはグローバル変数から取得
)

# このコードは、指定したモデルをHugging Faceからダウンロードし、指定したディレクトリに保存します。
# 既存のモデルファイルがあれば削除し、新しいモデル用のディレクトリを作成します。

モデルの重みは`/kaggle/tmp/model/`に保存されていることが確認できます。

In [ ]:
# 保存されたモデルの重みを確認するために、指定したディレクトリ内のファイル一覧を表示します。
!ls -l /kaggle/tmp/model

# 上記のコマンドは、'/kaggle/tmp/model'ディレクトリにあるファイルやフォルダの詳細情報（権限、サイズ、最終更新日など）をリスト表示します。

# ヘルパユーティリティファイル

これらは、vLLMサーバーを起動するために使用するヘルパー関数です。

In [ ]:
%%writefile util.py

# vLLMサーバーを起動するためのヘルパー

import subprocess
import os
import socket
import time

# 指定したポートが使用中かどうかをチェックする関数
def check_port(port: int) -> bool:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)  # 1秒以内に応答がない場合はタイムアウト
            result = sock.connect_ex(('localhost', port))  # ローカルホストの指定ポートに接続を試みる
            if result == 0:  # 結果が0ならポートは開いている
                return True
    except socket.error:
        pass  # ソケットエラーが発生した場合は無視
    
    return False  # ポートが開いていない場合はFalseを返す

# コマンドを実行し、指定したポートが開くのを待つ関数
def run_and_wait_for_port(
    cmd: list[str],  # 実行するコマンドのリスト
    port: int,  # チェックするポート番号
    env: dict[str, str] | None,  # 環境変数
    timeout: int = 60,  # タイムアウト時間（秒）
    debug: bool = False,  # デバッグモードのフラグ
) -> subprocess.Popen:  # subprocess.Popenオブジェクトを返す

    if check_port(port):  # ポートがすでに開いている場合
        raise ValueError(f"Port {port} is already open")  # エラーを発生させる
        
    # コマンドを非同期で実行
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},  # 環境変数を結合
        stdout=subprocess.DEVNULL if not debug else None,  # デバッグモードでない場合は標準出力を無視
        stderr=subprocess.DEVNULL if not debug else None,  # デバッグモードでない場合はエラー出力を無視
    )
    
    start_time = time.time()  # 開始時間を記録
    while time.time() - start_time < timeout:  # タイムアウトまでループ
        if check_port(port):  # ポートが開いたら
            return popen  # Popenオブジェクトを返す
        time.sleep(1)  # 1秒待つ
    
    popen.terminate()  # タイムアウトした場合はプロセスを終了
    raise Exception(f"Process did not open port {port} within {timeout} seconds.")  # エラーを発生させる

# 上記のコードは、vLLMサーバーを起動し、指定したポートが開くのを待つためのヘルパー関数を定義しています。
# これらの関数を使うことで、サーバーの起動状況を確認し、問題がなければ次の処理に進むことができます。

# vLLMサーバーをテストのために起動する

私たちのモデルはvLLMサーバーを使用してホスティングされます。以下では、Kaggle環境でどのように動作するかを理解するためにノートブックを起動します。

In [ ]:
# vLLMのパスと設定。

import importlib
from pathlib import Path
import util

# utilモジュールを再読み込みします。
util = importlib.reload(util)

# vLLMライブラリのパスを設定
g_srvlib_path = Path("/kaggle/tmp/srvlib")
# 指定したパスが存在することを確認
assert g_srvlib_path.exists()

# モデルのパスを設定
g_model_path = Path("/kaggle/tmp/model")
# 指定したパスが存在することを確認
assert g_model_path.exists()

# vLLMサーバーで使用するポート番号とモデル名を設定
g_vllm_port = 9999  # 使用するポート番号
g_vllm_model_name = "custom"  # モデル名

# このコードは、vLLMサーバーを運用するために必要なパスと設定を定義しています。
# これにより、サーバーの起動時やモデルの読み込み時に適切なリソースを指定できるようになります。

In [ ]:
# subprocessを使用してvLLMサーバーを実行
vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",  # vLLMのAPIサーバーをモジュールとして起動
    "--enforce-eager",  # イagerモードを強制するオプション
    "--model", str(g_model_path),  # 使用するモデルのパスを指定
    "--port", str(g_vllm_port),  # 使用するポート番号を指定
    "--served-model-name", g_vllm_model_name,  # 提供されるモデルの名前
    "--dtype=half"  # データ型をhalf精度に設定
],
    g_vllm_port,  # ポート番号を指定
    {"PYTHONPATH": str(g_srvlib_path)},  # PYTHONPATHにsrvlibのパスを追加
    debug=False  # デバッグモードを無効にする
)

print("vLLMが起動しました")  # サーバーが正常に起動したことを通知するメッセージ

llama3モデルが最初のTesla T4 GPUにロードされていることが確認できます。

In [ ]:
# GPUの状態を確認するために、nvidia-smiコマンドを実行します。
!nvidia-smi

# このコマンドは、NVIDIAのGPUの使用状況やメモリの状態、実行中のプロセスを表示します。
# モデルが適切にGPUにロードされていることを確認するために使用します。

## モデルの検証

最初のriggingジェネレーターを作成しましょう。riggingでは、ジェネレーターは強力なLLMパイプラインを作成するための基盤となります。

In [ ]:
# Riggingに接続する

import sys
import logging

# ライブラリのパスをシステムパスに追加
sys.path.insert(0, "/kaggle/tmp/lib")

# LiteLLMのログレベルを警告に設定
logging.getLogger("LiteLLM").setLevel(logging.WARNING)

import rigging as rg

# ジェネレーターを取得する
generator = rg.get_generator(
    f"openai/{g_vllm_model_name},"  # vLLMモデルの名前
    f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIベースURL
    "api_key=sk-1234,"  # APIキー（ダミーの例です）
    "stop=<|eot_id|>"  # Llamaモデルにはいくつかの設定が必要
)

# ジェネレーターにチャットメッセージを送信
answer = await generator.chat("Say Hello!").run()

print()
print('[Rigging Chat]')
print(type(answer), answer)  # 応答のタイプと内容を表示

print()
print('[LLM Response Only]')
print(type(answer.last), answer.last)  # 最後の応答のタイプと内容を表示

print()
answer_string = answer.last.content  # 最後の応答の内容を取得
print('[LLM Response as a String]')
print(answer.last.content)  # 最後の応答の内容を表示

# 上記のコードは、riggingを使用してvLLMサーバーに接続し、シンプルなチャット応答を取得するプロセスを示しています。
# ユーザーからの入力に対してモデルがどのように応答するかを検証します。

## 結果をpandasデータフレームに変換

`to_df()`メソッドを使用することで、チャット履歴を簡単にpandasデータフレームに変換することができます。

In [ ]:
# チャット応答をpandasデータフレームに変換する
answer_df = answer.to_df()

# データフレームの内容を表示する
answer_df

# 上記のコードは、チャットの応答をデータフレーム形式に変換し、内容を表示します。
# これにより、応答履歴をより構造化された形式で管理できます。

## モデルパラメータの変更

データベース接続文字列と同様に、Riggingのジェネレーターは、どのプロバイダー、モデル、APIキー、生成パラメータなどを使用するべきかを定義する文字列として表現できます。フォーマットは以下の通りです：

```
<provider>!<model>,<**kwargs>
```

ここでは、追加パラメータを使用してモデルをロードする例を示します：
- temperature=0.9
- max_tokens=512

これらの詳細については、こちらのドキュメントを参照してください: https://rigging.dreadnode.io/topics/generators/#overload-generation-params

In [ ]:
# モデルを追加パラメータでロードするジェネレーターを取得
generator = rg.get_generator(
    f"openai/{g_vllm_model_name},"  # vLLMモデルの名前
    f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIベースURL
    "api_key=sk-1234,"  # APIキー（ダミーの例です）
    "temperature=0.9,max_tokens=512,"  # 生成時の温度と最大トークン数の設定
    "stop=<|eot_id|>"  # Llamaモデルにはいくつかの設定が必要
)

# 上記のコードは、モデルを指定したパラメータ（temperatureやmax_tokensなど）でロードするためのジェネレーターを取得します。
# この設定により、生成される応答の性質を調整することができます。

別の方法として、`rg.GenerateParams`クラスを使用してこれらのパラメータを設定することもできます。このクラスでは、さまざまなモデルパラメータを設定することができます：

```
rg.GenerateParams(
    *,
    temperature: float | None = None,  # 温度
    max_tokens: int | None = None,  # 最大トークン数
    top_k: int | None = None,  # トップkフィルタリング
    top_p: float | None = None,  # トップpサンプリング
    stop: list[str] | None = None,  # 停止トークンのリスト
    presence_penalty: float | None = None,  # プレゼンスペナルティ
    frequency_penalty: float | None = None,  # 頻度ペナルティ
    api_base: str | None = None,  # APIのベースURL
    timeout: int | None = None,  # タイムアウト時間
    seed: int | None = None,  # シード値
    extra: dict[str, typing.Any] = None,  # その他の追加パラメータ
)
```

https://rigging.dreadnode.io/api/generator/#rigging.generator.GenerateParams

In [ ]:
# GenerateParamsを使用してモデルパラメータを設定
rg_params = rg.GenerateParams(
    temperature = 0.9,  # 温度を0.9に設定
    max_tokens = 512,  # 最大トークン数を512に設定
)

# ジェネレーターに対してチャットを開始し、パラメータを適用
base_chat = generator.chat(params=rg_params)

# 新たなプロンプトでチャットを実行
answer = await base_chat.fork('How is it going?').run()

# 最後の応答の内容を表示
print(answer.last.content)

# 上記のコードは、rg.GenerateParamsを使用して設定したモデルパラメータを使って、
# チャットを実行し、特定のプロンプトに対する応答を取得するプロセスを示しています。

また、パラメータはチェーン内で`params`を使って設定することもできます。

In [ ]:
# チェーン内でパラメータを設定してチャットを実行
base_chat = generator.chat()  # パラメータは設定しない

# 新たなプロンプトでチャットを実行し、パラメータを指定
answer = await base_chat.fork('How is it going?') \
    .with_(temperature = 0.9, max_tokens = 512) \  # 温度と最大トークン数を設定
    .run()

# 最後の応答の内容を表示
print(answer.last.content)

# 上記のコードは、パラメータを直接チェーン内で設定し、
# チャットを実行する方法を示しています。これにより、より柔軟に生成パラメータを指定できます。

# パースされた出力の例

次に、以下のようなパイプラインを作成します：
1. `Answer`というriggingモデルを作成します。これにより、モデルの結果からパースする期待される出力を説明します。
    - 出力が`yes`または`no`のいずれかであることを確認するバリデーターを追加します。
    - これは完全にカスタマイズ可能です。
    - ここでの`validate_content`は、応答が期待される形式（小文字で「yes」または「no」で始まる）に準拠していることを保証します。
2. プロンプト内で`Answer.xml_example()`を使用して、LLMに出力の期待する形式を伝えます。
3. 後で`.until_parsed_as(Answer)`を使用して、ここで定義されたとおりにLLMの出力が抽出されることを確保します。

**注意**: `until_parsed_as()`は、デフォルトで5の`max_rounds`パラメータを受け取ることができます。

In [ ]:
import typing as t
from pydantic import field_validator

# モデルAnswerを定義
class Answer(rg.Model):
    content: t.Literal["yes", "no"]  # 出力が"yes"または"no"であることを指定

    # contentフィールドのバリデーションを行うメソッド
    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:  # 有効な応答と比較
            if v.lower().startswith(valid):  # 応答が小文字で"yes"または"no"で始まるか確認
                return valid  # 有効な場合、その値を返す
        raise ValueError("Invalid answer, must be 'yes' or 'no'")  # 無効な場合はエラーを発生

    # XML例を提供するクラスメソッド
    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"  # XML形式の例を返す

# 上記のコードは、期待される出力として"yes"または"no"を定義し、
# それに基づいて応答をバリデーションするモデルAnswerを作成します。

In [ ]:
# xml_exampleメソッドを使用して、XML形式の例を確認します。
Answer.xml_example() 

# このコードは、モデルAnswerのXML例を表示し、プロンプトに使用できる期待される出力の形式を確認します。

In [ ]:
# ジェネレーターを設定し、プロンプトを定義
generator = rg.get_generator(
    f"openai/{g_vllm_model_name},"  # vLLMモデルの名前
    f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIベースURL
    "api_key=sk-1234,"  # APIキー（ダミーの例です）
    "temperature=1.2"  # 温度を1.2に設定
    # "stop=<|eot_id|>"  # Llamaモデルにはいくつかの設定が必要
)

# キーワード、カテゴリ、最後の質問を設定
keyword='Tom Hanks'
category='Famous Person'
last_question='Is it a famous person?'

# プロンプトを定義
prompt = f"""\
            このゲームの秘密の言葉は「{keyword}」です [{category}]

            あなたは現在、上記の言葉についての質問に答えています。

            次の質問は「{last_question}」です。

            上のyes/noの質問に答え、次の形式で記入してください：
            {Answer.xml_example()}

            - あなたの回答は上記のキーワードに基づき正確であるべきです
            - いつも「yes」または「no」で答えてください

            答えは何ですか？
"""

# チャットを実行し、パースされた応答を取得
chat = await (
    generator
    .chat(prompt)  # プロンプトに基づくチャットを開始
    .until_parsed_as(Answer, max_rounds=50)  # パースされた出力を取得するまで実行
    .run()
)

# フルチャット履歴を表示
print('=== 完全チャット ===')
print(chat)

print()
# LLMの応答のみを表示
print('=== LLM応答のみ ===')
print(chat.last)

print()
# パースされた答えを表示
print('=== パースされた回答 ===')
print(chat.last.parse(Answer).content)  # パースした回答の内容を表示

# 上記のコードは、指定したプロンプトに基づきLLMに質問を送り、応答を取得します。
# その後、応答をパースして期待する形式に確認するプロセスを示しています。

# Riggingを使用した例の質問者チャットパイプラインを作成

次に、キーワードが何であるかを判断するための質問者パイプラインを作成しましょう。

最初に、出力をパースするために使用する`Question`オブジェクトを作成します。

In [ ]:
from pydantic import StringConstraints  # noqa

# 空白をトリムするための型アノテーションを定義
str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

# Questionモデルを定義
class Question(rg.Model):
    content: str_strip  # 質問の内容（空白をトリムする）

    # XMLの例を提供するクラスメソッド
    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()  # XML形式で質問の例を返す

# 上記のコードは、質問をパースするためのQuestionモデルを定義し、
# 質問の内容が空白をトリムされることを保証します。また、XML形式の例も提供します。

In [ ]:
# 質問者のチャットを設定し、キーワードについて質問を行う
base = generator.chat("""\
あなたは20の質問ゲームの才能あるプレイヤーです。あなたは正確で、集中力があり、
構造的なアプローチを持っています。あなたは有用な質問を作成し、推測を行うか、
キーワードに関する質問に答えます。
""")

# 次の質問を行うためのチャットを実行
question_chat = await (base.fork(
    f"""\
あなたは現在、次の質問を行っています。

質問を作成し、次の形式で記入してください：
{Question.xml_example()}

- あなたの応答は、最も多くの情報を収集する集中した質問であるべきです
- 質問を一般的に始めてください
- 残りの検索空間を二分することを常に試みてください
- 過去の質問と回答に注意を払ってください

次の質問は何ですか？
"""
)
.until_parsed_as(Question, attempt_recovery=True)  # 質問をパースし、回復を試みる
.run()
)

# 上記のコードは、質問者が未来の質問に基づいてゲームを進めるためのチャットを実行します。
# それにより、キーワードに関する情報を収集するための集中した質問が生成されます。

In [ ]:
# 会話のデータフレーム表現を表示
question_chat.to_df()

# このコードは、質問チャットの履歴をpandasデータフレームとして表示します。
# これにより、会話の進行状況や内容を構造化された形式で確認できます。

私たちは現在、LLMの応答が質問を含んでいることが確信でき、それに基づいて質問をパースすることができます。

In [ ]:
# 最後のチャットの応答をQuestionモデルを用いてパースし、質問を取得
question = question_chat.last.parse(Question).content
print(question)  # パースした質問を表示

# 上記のコードは、最新の質問チャットから質問を抽出し、それを表示します。
# これにより、LLMが生成した質問内容を確認できます。

# キーワードデータフレームの作成
**注意**: これは公開セットの可能なキーワードを知っているからこそ機能します。最終リーダーボードでは動作しません。

In [ ]:
# キーワードデータを取得するために、指定したURLからスクリプトをダウンロード
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

# このコマンドは、Kaggleのリポジトリからキーワード定義スクリプトをダウンロードし、
# `keywords_local.py`という名前のファイルとして保存します。これにより、キーワードのリストにアクセスできます。

In [ ]:
# ダウンロードしたキーワードファイルの先頭部分を表示
!head keywords_local.py

# このコマンドは、`keywords_local.py`ファイルの最初の数行を表示し、ファイルの内容やキーワードのリストを確認します。

In [ ]:
import sys
import json
import pandas as pd

# カレントディレクトリにパスを追加
sys.path.append('./')

# KEYWORDS_JSONをインポート
from keywords_local import KEYWORDS_JSON

# テキストの最初の単語を大文字に変換する関数
def capitalize_first_word(text):
    if not text:
        return text
    return text[0].upper() + text[1:].lower()

# キーワードデータフレームを作成する関数
def create_keyword_df(KEYWORDS_JSON):
    keywords_dict = json.loads(KEYWORDS_JSON)  # JSON形式のキーワードデータを辞書に変換

    category_words_dict = {}
    all_words = []
    all_cat_words = []
    
    for d in keywords_dict:  # 辞書内の各項目をループ
        words = [w['keyword'] for w in d['words']]  # キーワードを抽出
        cat_word = [(d['category'], w['keyword']) for w in d['words']]  # カテゴリとキーワードのペアを作成
        category_words_dict[d['category']] = words
        all_words += words  # すべてのキーワードを追加
        all_cat_words += cat_word  # すべてのカテゴリ・キーワードペアを追加

    # データフレームを作成
    keyword_df = pd.DataFrame(all_cat_words, columns=['category', 'keyword'])
    keyword_df['first_letter'] = keyword_df['keyword'].str[0]  # 最初の文字を抽出
    keyword_df['second_letter'] = keyword_df['keyword'].str[1]  # 2番目の文字を抽出
    
    # データフレームをparquet形式で保存
    keyword_df.to_parquet('keywords.parquet')
    
# キーワードデータフレームを作成する関数を実行
create_keyword_df(KEYWORDS_JSON)

# 上記のコードは、JSONデータからキーワードデータフレームを作成し、parquetファイルとして保存します。
# これにより、キーワードのカテゴリとそれに対応する単語の情報を管理できるようになります。

In [ ]:
# 作成したキーワードデータフレームをparquetファイルから読み込み、サンプルを表示
keywords_df = pd.read_parquet('keywords.parquet')
keywords_df.sample(10)  # ランダムに10行を表示

# このコードは、キーワードデータフレームを読み込み、その内容の一部をサンプルとして表示します。
# これにより、データフレームの構造とキーワードのカテゴリ及びその内容を確認できます。

In [ ]:
# 各カテゴリのキーワードのカウントを表示
keywords_df['category'].value_counts()

# このコードは、キーワードデータフレーム内の各カテゴリに含まれるキーワードの数をカウントし、
# その結果を表示します。これにより、各カテゴリのキーワードの分布を確認できます。

# 最終提出用の`main.py`スクリプトを作成

私たちの最終提出は、`main`ファイルを含む圧縮ディレクトリになります。以下にそのファイルを示します。

In [ ]:
%%writefile main.py

# メインエージェントファイル

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

import string
import numpy as np
import pandas as pd

# パスの修正

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"
    g_model_path = g_agent_path / "model"
    g_srvlib_path = g_agent_path / "srvlib"
else:
    g_agent_path = Path('/kaggle/working')
    
sys.path.insert(0, str(g_lib_path))

# ロギングのノイズを抑える

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# 固定インポート

import util # noqa
import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# 定数

g_vllm_port = 9999
g_vllm_model_name = "custom"

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
)

# 型定義

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("Invalid answer, must be 'yes' or 'no'")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="question").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="thing/place").to_pretty_xml()


# 関数

async def ask(base: rg.ChatPipeline, observation: Observation) -> str:
    if observation.step == 0:
        # 最初の質問のバグ修正のためオーバーライド
        return "Are we playing 20 questions?"
    
    
    full_question = f"""\
                あなたは現在、次の質問を行っています。

                <game-history>
                {observation.get_history_as_xml(skip_guesses=True)}
                </game-history>

                上記の履歴に基づき、次の最も有用なyes/noの質問を行い、
                次の形式で記入してください：
                {Question.xml_example()}

                - あなたの応答は、最も多くの情報を収集する集中した質問であるべきです
                - 質問を一般的に始めてください
                - 残りの検索空間を二分することを常に試みてください
                - 過去の質問と回答に注意を払ってください

                次の質問は何ですか？
                """
    
    print(' ======質問中 ======')
    print(full_question)
    
    
    try:
        chat = await (
             base.fork(full_question)
            .until_parsed_as(Question, attempt_recovery=True, max_rounds=20)
            .run()
        )
        return chat.last.parse(Question).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'Is it a person?'

async def answer(base: rg.ChatPipeline, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("Keyword wasn't provided to answerer", file=sys.stderr)
        return "yes" # バグ修正のためオーバーライド
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            # 5回ループして最も頻繁な応答を取得
            chat = await (
                base.fork(
                    f"""
                    キーワード: [{observation.keyword}]

                    質問: {last_question}

                    yes または no で答え、形式は次の通りにしてください：<answer>yes</answer> または <answer>no</answer>
                    """
                )
                .until_parsed_as(Answer, attempt_recovery=True, max_rounds=20)
                .run()
            )
            responses.append(chat.last.parse(Answer).content.strip('*'))
            
        print(f'応答は {responses}')
        return pd.Series(responses).value_counts().index[0]
    except rg.error.MessagesExhaustedMaxRoundsError:
        print('%%%%%%%%%%%% エラーが発生したためyesと回答します %%%%%%%%%%%% ')
        return 'yes'

async def guess(base: rg.ChatPipeline, observation: Observation) -> str:
    try:

        chat = await (
            base.fork(
                f"""\
                あなたは現在、キーワードの情報に基づいて推測を行っています。

                <game-history>
                {observation.get_history_as_xml()}
                </game-history>

                上記の履歴に基づき、キーワードに対する次の最良の推測を生成し、
                次の形式で記入してください：
                {Guess.xml_example()}

                - 上記の履歴に基づいて重複する推測を避けてください
                - 推測は特定の人、場所、または物であるべきです

                あなたの推測は何ですか？
                """
            )
            .until_parsed_as(Guess, attempt_recovery=True, max_rounds=20)
            .run()
        )

        return chat.last.parse(Guess).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'france'
    
# vLLMとジェネレーターの構成

try:
    vllm = util.run_and_wait_for_port([
        "python", "-m",
        "vllm.entrypoints.openai.api_server",
        "--enforce-eager",
        "--model", str(g_model_path),
        "--port", str(g_vllm_port),
        "--served-model-name", g_vllm_model_name,
        "--dtype=half"
    ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

    print("vLLMの起動完了")
except ValueError:
    print('vLLMは既に起動しています')
    
    
generator = rg.get_generator(g_generator_id)

base =  generator.chat("""\
あなたは20の質問ゲームの才能あるプレイヤーです。あなたは正確で、集中力があり、
構造的なアプローチを持っています。あなたは有用な質問を作成し、推測を行い、または
キーワードに関する質問に答えます。

""")

# エントリポイント
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは'{letters[0]}'の文字で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次の文字のいずれかで始まりますか {formatted_letters}?"

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

# シンプルな質問者
class SimpleQuestionerAgent():
    def __init__(self, keyword_df: pd.DataFrame):
        self.keyword_df = keyword_df
        self.keyword_df_init = keyword_df.copy()
        self.round = 0
        self.category_questions = [
            "20の質問ゲームをしていますか？",
            "キーワードは物で、場所ではありませんか？",
            "キーワードは場所ですか？",
        ]
        self.found_category = False
        
    def filter_keywords(self, obs):
        print(self.keyword_df.shape)
        # 過去の回答に基づいてkeyword_dfをフィルタリング
        for i, answer in enumerate(obs.answers):
            if obs.questions[i] in self.category_questions:
                if answer == 'yes':
                    if obs.questions[i] == "キーワードは物で、場所ではありませんか？":
                        self.found_category = 'things'
                    if obs.questions[i] == "キーワードは場所ですか？":
                        self.found_category = 'place'
                    fc = self.found_category
                    self.keyword_df = self.keyword_df.query('category == @fc').reset_index(drop=True)
    
            if obs.questions[i].startswith('キーワードは次の文字で始まりますか？'):
                if self.keyword_df['first_letter'].nunique() <= 1:
                    break
                letter_question = obs.questions[i]
                letters = extract_letters_from_question(letter_question)
                self.keyword_df = self.keyword_df.reset_index(drop=True).copy()
                if obs.answers[i] == 'yes':
                    self.keyword_df = self.keyword_df.loc[
                        self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
                elif obs.answers[i] == 'no':
                    self.keyword_df = self.keyword_df.loc[
                        ~self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
            
    def get_letters(self, obs, max_letters=20):
        n_letters = self.keyword_df['first_letter'].nunique()
        sample_letters = self.keyword_df['first_letter'].drop_duplicates().sample(n_letters // 2).values.tolist()
        sample_letters = sample_letters[:max_letters]
        print('サンプル文字', n_letters, sample_letters)
        return sample_letters # ', '.join(sample_letters)
    
    def __call__(self, obs, *args):
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
        self.filter_keywords(obs)
        if obs.turnType == 'ask':
            self.round += 1
            if (self.round <= 3 and not self.found_category):
                response = self.category_questions[self.round - 1]
            else:
                sample_letters = self.get_letters(obs)
                if len(sample_letters) == 0:
                    n_sample = min(len(self.keyword_df), 10)
                    possible_keywords = ", ".join(self.keyword_df['keyword'].sample(n_sample).values.tolist())
                    response = f"次の中のキーワードはどれですか？ {possible_keywords}"
                else:
                    sample_letters_str = str(sample_letters).replace('[','').replace(']','')
                    response = format_first_letter_question(sample_letters)
        elif obs.turnType == 'guess':
            response = self.keyword_df['keyword'].sample(1).values[0]
            # 推測した単語を削除
            updated_df = self.keyword_df.loc[self.keyword_df['keyword'] != response].reset_index(drop=True).copy()
            if len(updated_df) >= 1:
                self.keyword_df = updated_df.copy()
            else:
                self.keyword_df = self.keyword_df_init.copy() # データフレームをリセット
        return response


keyword_df = pd.read_parquet(f'{g_agent_path}/keywords.parquet')
question_agent = None

async def observe(obs: t.Any) -> str:
    observation = Observation(**obs.__dict__)
    global question_agent
    if question_agent is None:
        question_agent = SimpleQuestionerAgent(keyword_df)

    try:
        match observation.turnType:
            case "ask":
                return question_agent(obs)
            case "answer":
                return await answer(base, observation)
            case "guess":
                return question_agent(obs)

            case _:
                raise ValueError("Unknown turn type")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    # フレームワーク内で非同期を処理
    import asyncio
    return asyncio.run(observe(obs))

# エージェントを自己対戦させてテストする

In [ ]:
# 最初の文字の質問をフォーマットする関数を定義
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは'{letters[0]}'の文字で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次の文字のいずれかで始まりますか {formatted_letters}?"

# 例として'a', 'b', 'c'の場合にフォーマットされた質問を表示
format_first_letter_question(['a', 'b', 'c'])

import re

# 質問から文字を抽出する関数を定義
def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"  # アポストロフィで囲まれた文字を検索
    matches = re.findall(pattern, question)  # 正規表現を使ってマッチを抽出
    return matches

# 上記のコードは、キーワードの最初の文字に関する質問をフォーマットする方法を示し、
# また、特定の質問から文字を抽出するための関数を定義しています。

In [ ]:
# 自動再読み込みを有効にする
%load_ext autoreload
%autoreload 2

# main.pyから必要なクラスと関数をインポート
from main import Observation, agent_fn, observe

# 上記のコードは、Jupyter環境においてモジュールが変更された際に自動的に再読み込みを行い、
# 最新の定義をインポートするための設定をしています。これにより、開発中のコード変更が即座に反映されます。

In [ ]:
# vLLMが実行中かどうかを確認
!ps -aef | grep vllm

# このコマンドは、現在実行中のプロセスの中からvLLMに関連するプロセスをフィルタリングして表示します。
# これにより、vLLMが正しく起動しているか確認できます。

In [ ]:
import pandas as pd

# キーワードデータフレームをparquetファイルから読み込む
keyword_df = pd.read_parquet('keywords.parquet')
# ランダムに1つのサンプルを取得
sample = keyword_df.sample(1)

# 観測データを作成
obs = Observation(step=0,
    role='guesser',  # 役割を'guesser'に設定
    turnType="ask",  # 最初は質問を行う
    keyword=sample['keyword'].values[0],  # サンプルからキーワードを取得
    category=sample['category'].values[0],  # サンプルからカテゴリを取得
    questions=[],  # 質問のリストを初期化
    answers=[],  # 回答のリストを初期化
    guesses=[],  # 推測のリストを初期化
)

question_agent = None

# 20ラウンドにわたって質問と回答を繰り返す
for i in range(20):
    obs.role = 'guesser'
    obs.turnType = 'ask'
    question = await observe(obs)  # 質問を生成
    print(f'[{i} Question]: {question}')
    obs.questions.append(question)  # 質問を追加
    obs.role = 'answerer'
    obs.turnType = 'answer'
    answer = await observe(obs)  # 回答を生成
    obs.answers.append(answer)  # 回答を追加
    
    # 各質問に対する正しい回答を生成
    if obs.questions[-1].startswith('Are we playing 20 questions?'):
        gt_answer = answer  # 任意の回答を記録
    elif obs.questions[-1].startswith('Is the keyword a thing that is not a location?'):
        if sample['category'].values[0] == 'things':
            gt_answer = 'yes'  # 正解
        else:
            gt_answer = 'no'  # 不正解
    elif obs.questions[-1].startswith('Is the keyword a place?'):
        if sample['category'].values[0] == 'place':
            gt_answer = 'yes'  # 正解
        else:
            gt_answer = 'no'  # 不正解
    elif obs.questions[-1].startswith('Does the keyword start'):
        letters_guess = extract_letters_from_question(obs.questions[-1])  # 質問から文字を抽出
        gt_answer = obs.keyword[0] in letters_guess
        gt_answer = 'yes' if gt_answer else 'no'  # 正解かどうかを判断
    elif obs.questions[-1].startswith('Is the keyword one of the following?'):
        possible_kw = obs.questions[-1].replace('Is the keyword one of the following? ', '').split(',')
        possible_kw = [c.strip(' ') for c in possible_kw]  # キーワードのリストを整形
        print(possible_kw)
        gt_answer = obs.keyword in possible_kw  # サンプルキーワードが候補に含まれているか
        gt_answer = 'yes' if gt_answer else 'no'  # 正解かどうかを判断

    print(f'[{i} Answer]: {answer} [True Answer]: {gt_answer}')
    if answer != gt_answer:  # 回答が正しいか確認
        break

    obs.role = 'guesser'
    obs.turnType = 'guess'  # 推測を行うターンに変更
    guess = await observe(obs)  # 推測を生成
    print(f'[{i} Guess]: {guess} - [Keyword]: {obs.keyword}')
    obs.guesses.append(guess)  # 推測を追加
    if guess == obs.keyword:  # 推測が正解か確認
        print('GOT IT!')
        break
        
    obs.step += 1  # ステップを進める

# 上記のコードは、キーワードについて20ターンにわたって質問、回答、推測を繰り返すエージェントの動作をシミュレーションします。

In [ ]:
# 最後の質問を定義
last_question = "Does the keyword start with one of the letters 'a', 'n' or 'g'?"
# ジェネレーターを使用して答えるプロンプトを構築
out = await generator.chat(f"""\
あなたは、キーワード[{obs.keyword}]に対して正確にyes/noの質問に答えます。このキーワードはカテゴリ[{obs.category}]に属します。

[QUESTION] "{last_question}" [/QUESTION]

キーワードは[{obs.keyword}]です - 質問にはこの特定のキーワードに正確に回答してください。

上のyes/noの質問に答え、次の形式で記入してください：
{Answer.xml_example()}
""").run() 

# 上記のコードは、ジェネレーターを使用して特定の質問に対するyes/noの応答を生成するためのプロンプトを作成します。
# これは、エージェントがキーワードに基づいて正確な回答を提供するのを助けるためのものです。

In [ ]:
# 最後の質問を定義
last_question = "Does the keyword start with one of the letters 'a', 'n' or 'g'?"
# ジェネレーターを使用して答えるプロンプトを構築
out = await generator.chat(f"""\
あなたは20の質問ゲームをプレイしています。あなたのタスクは、特定のキーワードに関するyes/noの質問に正確に答えることです。

キーワード: [{obs.keyword}]
カテゴリ: [{obs.category}]

質問: {last_question}

指示:
1. 回答する際は、キーワード[{obs.keyword}]のみを考慮してください。
2. 質問には「yes」または「no」で答えてください。
3. 答えを出す前に、正確性をダブルチェックしてください。
4. 下記のXML形式で答えを提供してください。

あなたの応答はこの正確な形式であるべきです：
<answer>yes</answer>
OR
<answer>no</answer>

さて、キーワード[{obs.keyword}]に対して質問に正確に答えてください：
""").run() 

# 上記のコードは、ジェネレーターを使用してプレイヤーに特定の質問に対するyes/noの応答を正確に生成するよう指示するプロンプトを作成します。

In [ ]:
# 最後の出力の内容を表示
print(out.prev[-1].content)

# このコードは、生成された応答の前のやり取りの最後の内容を表示します。
# これにより、LLMがどのように応答したかを確認することができます。

In [ ]:
# 最後の出力の内容を表示
out.last

# このコードは、生成された応答の最後の内容を表示します。
# これにより、LLMの応答の詳細を確認することができます。

# モデルとコードを提出用に圧縮



In [ ]:
# pigzとpvをインストールします
!apt install pigz pv

# このコマンドは、圧縮を高速化するためのpigz（並列実行のgzip）と
# プログレスバーを表示するためのpvをインストールします。

In [ ]:
# モデルとコードを圧縮して提出用のtar.gzファイルを作成します
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \  # 指定したディレクトリから必要なフォルダを追加
    -C /kaggle/working main.py util.py \  # メインのPythonファイルとユーティリティファイルを追加
    -C /kaggle/working keywords.parquet  # キーワードデータファイルを追加

# このコマンドは、指定されたファイルとディレクトリを圧縮し、提出用のtar.gzファイルを作成します。

In [ ]:
# 作成したsubmission.tar.gzファイルを表示
!ls -GFlash --color

# このコマンドは、カラフルな表示で現在のディレクトリ内のファイルとフォルダをリストします。
# これにより、提出用に圧縮したファイルが正しく作成されたか確認できます。

# Kaggle CLIを使用しての提出

必要に応じて、ノートブックを再実行することなく、KaggleのCLIインターフェースを使用して提出することができます。

In [ ]:
# Kaggle CLIを使用してコンペティションに提出するコマンド（コメントアウトされています）
# !KAGGLE_USERNAME={KAGGLE_USERNAME} \
#  KAGGLE_KEY={KAGGLE_KEY} \
#  kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "submit from notebook"

# 上記のコマンドは、KaggleのCLIを使用してコンペティションに提出するためのものです。
# KAGGLE_USERNAMEとKAGGLE_KEYは、Kaggleの認証情報を指定するために使用します。
# submission.tar.gzファイルを提出し、メッセージを添えて送信します。